![](http://i.postimg.cc/BbD7fnL5/Screenshot-2025-11-04-201307.png)

# 1.IMPORTS


In [6]:
# ===============================
# 1. IMPORTS
# ===============================
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Subtract
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

import re
import nltk
import spacy
import seaborn as sns
from tqdm import tqdm
from tqdm.notebook import tqdm
from nltk.corpus import stopwords

# Converting Text file to csv

In [7]:
# Read data from text
with open('/kaggle/input/mit-plagairism-detection-dataset/train_snli.txt') as file:
    data = file.readlines()
    
# prepare csv file
with open('data.csv', 'w', newline= '') as csvfile:
    filenames = ['source_txt', 'plagiarism_txt', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=filenames)
    
    writer.writeheader()
    for line in tqdm(data):
        parts = line.strip().split('\t')
        source_txt = parts[0]
        plagiarishm_txt = parts[1]
        label = int(parts[2])
        
        writer.writerow({
            'source_txt' : source_txt,
            'plagiarism_txt' : plagiarishm_txt,
            'label' : label
        })
print('CSV file created successfully...')

  0%|          | 0/367373 [00:00<?, ?it/s]

CSV file created successfully...


# 2. LOAD DATA


In [8]:

# Replace with your dataset path
df = pd.read_csv("data.csv")
df.head()

,source_txt,plagiarism_txt,label
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [9]:
df.duplicated().sum()


454

In [10]:
df.drop_duplicates(inplace=True)


# 3. BASIC CLEANING FUNCTION


In [11]:

import re, string


def clean_text(text):
    text = text.lower()
    text = re.sub(r"\n", " ", text) #Replace newlines with spaces
    text = text.translate(str.maketrans("", "", string.punctuation)) #Remove punctuation
    return text


# Apply cleaning
df["source_clean"] = df["source_txt"].astype(str).apply(clean_text)
df["plag_clean"] = df["plagiarism_txt"].astype(str).apply(clean_text)

# 4. TOKENIZATION


In [12]:

source = df['source_clean'].tolist()
plag = df['plag_clean'].tolist()
labels = df['label'].values


tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(source + plag)


source_seq = tokenizer.texts_to_sequences(source)
plag_seq = tokenizer.texts_to_sequences(plag)


max_len = 50
source_pad = pad_sequences(source_seq, maxlen=max_len)
plag_pad = pad_sequences(plag_seq, maxlen=max_len)

# 5. SPLIT DATA


In [13]:

X_train_src, X_test_src, X_train_plag, X_test_plag, y_train, y_test = train_test_split(
source_pad, plag_pad, labels, test_size=0.2, random_state=42)

# 6. BUILD LSTM MODEL


In [14]:
#Siamese LSTM network
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
lstm_dim = 64


input_a = Input(shape=(max_len,))
input_b = Input(shape=(max_len,))


embedding = Embedding(vocab_size, embedding_dim, input_length=max_len)
lstm = LSTM(lstm_dim)


encoded_a = lstm(embedding(input_a))
encoded_b = lstm(embedding(input_b))

# The idea: if the texts are similar, the difference vector should be small.
merged = Subtract()([encoded_a, encoded_b])
merged = Dense(64, activation="relu")(merged)
merged = Dropout(0.5)(merged)
out = Dense(1, activation="sigmoid")(merged)


model = Model(inputs=[input_a, input_b], outputs=out)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1762510247.328969      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │  3,874,432 │ input_layer[0][0… │
│ (Embedding)         │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract (Subtract) │ (None, 64)        │          0 │ lstm[0][0],       │
│                     │                   │            │ lstm[1][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ subtract[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,928,065 (14.98 MB)

 Trainable params: 3,928,065 (14.98 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)


# 7. TRAIN MODEL


In [ ]:

history = model.fit(
    [X_train_src, X_train_plag],
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

Epoch 1/10


I0000 00:00:1762510253.457593     104 cuda_dnn.cc:529] Loaded cuDNN version 90300


1905/7339 ━━━━━━━━━━━━━━━━━━━━ 52s 10ms/step - accuracy: 0.6889 - loss: 0.5644

# 8. EVALUATION


In [ ]:

pred = model.predict([X_test_src, X_test_plag])
pred_label = (pred > 0.5).astype(int)


print("Accuracy:", accuracy_score(y_test, pred_label))
print(classification_report(y_test, pred_label))

# 9. PLOT TRAINING CURVES


In [ ]:

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy over epochs')
plt.legend()
plt.show()


plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss over epochs')
plt.legend()
plt.show()

# 10. SAMPLE PREDICTION


In [ ]:

idx = 0
src = X_test_src[idx:idx+1]
plg = X_test_plag[idx:idx+1]
print("Pred:", model.predict([src, plg])[0][0])

# Testing Model on User Inputs

In [ ]:
def predict_pair(source_text, plag_text, tokenizer, max_len):
    # Convert to sequences
    src_seq = tokenizer.texts_to_sequences([source_text])
    plg_seq = tokenizer.texts_to_sequences([plag_text])
    
    # Pad
    src_pad = pad_sequences(src_seq, maxlen=max_len)
    plg_pad = pad_sequences(plg_seq, maxlen=max_len)

    # Predict
    pred = model.predict([src_pad, plg_pad])[0][0]

    # Show result
    print("Prediction score:", pred)
    if pred > 0.5:
        print("✅ Likely PLAGIARIZED")
    else:
        print("❌ Likely NOT plagiarized")

    return pred


In [ ]:
src = "A woman is standing outside wearing glasses."
plg = "A woman wearing glasses is outdoors."

predict_pair(src, plg, tokenizer, max_len)


In [ ]:
src = "Two kids are sleeping on a bench."
plg = "A car is driving on the road."

predict_pair(src, plg, tokenizer, max_len)


<span style="color: white; background-color: red; padding: 10px 20px; border-radius: 10px; font-size: 36px; font-weight: bold;">Smash That Upvote !! Thanks ! 🚀😃</span>
